In [ ]:
import os

if not os.getcwd().endswith("syftr"):
    os.chdir(os.path.dirname(os.getcwd()))
    print(f"Changed working directory to: {os.getcwd()}")

In [ ]:
import pandas as pd

pd.set_option("mode.chained_assignment", "raise")
pd.set_option("display.max_rows", 200)


In [ ]:
from syftr.plotting.insights import load_studies

study_names = [
    "judge-data1--test--crag_hf-music--music",
    "judge-data1--test--financebench_hf",
    "judge-data1--test--hotpotqa_hf-train_hard--train_hard",
    "judge-data1--test--multihoprag_hf",
]
df, study_stats_table, exceptions_table = load_studies(study_names, only_successful_trials=False)
display(df.T)


In [ ]:
df.iloc[0]['user_attrs_metric_eval_results']

In [ ]:
# Extract the QAR triplet and judge scores from each QA pair from each row
records = [
    record
    for row in df['user_attrs_metric_eval_results']
    for record in row.values()
]

new_df = pd.DataFrame(records)

new_df = new_df.rename(columns={"passing": "gpt-4o-passing", "raw_score": "gpt-4o-raw-score"})[["question", "answer", "response", "gpt-4o-passing", "gpt-4o-raw-score"]]
from sklearn.utils import shuffle

new_df = shuffle(new_df).reset_index(drop=True)
new_df

In [ ]:
new_df.to_csv("/tmp/qars.csv")

In [ ]:
labeled_df = pd.read_csv("/home/alex/Downloads/Human Eval Redux - qars (5).csv")
labeled_df = labeled_df[~pd.isna(labeled_df['Human Score'])]
labeled_df

In [ ]:
qar_template = """
## User Query
{query}

## Reference Answer
{reference_answer}

## Generated Answer
{generated_answer}
"""
labeled_df['qar_prompt'] = labeled_df.apply(
    lambda row: qar_template.format(
        query=row.question,
        reference_answer=row.answer,
        generated_answer=row.response,
    ),
    axis=1
)

labeled_df

In [ ]:
labeled_df.to_csv("qars_labeled.csv")

In [ ]:
import datasets

In [ ]:
dataset = datasets.Dataset.from_pandas(labeled_df)
dataset

In [ ]:
dataset.push_to_hub(repo_id='DataRobot-Research/judge-eval', commit_message="create datasets dataset")